#### 1. 卷积样例

In [1]:
import tensorflow as tf

In [2]:
# input输入图像是一个四维矩阵
# 第1维是第X个图像
# 第2第3维是长度宽度为32*32
# 第4维是RGB颜色通道数为3的矩阵
input = tf.placeholder(
    name='input',
    shape=[None, 32, 32, 3], 
    dtype=tf.float32
)

# filter是一个四维矩阵
# 第1第2维是过滤器尺寸width和height（5*5），
# 第3维当前层深度（3）
# 第4维是过滤器深度（16）
# 初始化为正态分布的随机变量
filter_weight = tf.get_variable(
    name='weights', 
    shape=[5, 5, 3, 16], 
    initializer=tf.truncated_normal_initializer(stddev=0.1)
)

# biases偏置量是一个一维矩阵
# 第1维是偏置量的深度（16）
# 初始化为数值都是0.1的一维矩阵
biases = tf.get_variable(
    name='biases', 
    shape=[16], 
    initializer=tf.constant_initializer(0.1)
)

# tf.nn.conv2d的strides步长参数
# strides步长是一个四维矩阵
# 第1维和第4维必须为1，因为卷积层的步长只对矩阵的长宽有效
# 第2维和第3维是长和宽的步长
# 
# tf.nn.conv2d的padding参数只提供两种SAME以及VAILD
# SAME表示添加0填充
# VAILD表示不添加填充
conv = tf.nn.conv2d(
    input=input, 
    filter=filter_weight, 
    strides=[1,1,1,1], 
    padding='SAME'
)

# tf.nn.bias_add提供了一个方便的函数给节点添加偏置量（注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置量）
# 例如下一层神经网络的大小为2*2，但是偏置量只有一个数（因为深度为1），而2*2矩阵的每一个项都需要加上这个偏置量。
bias = tf.nn.bias_add(value=conv, bias=biases)

# 把计算结果通过relu激活函数进行去线性化
actived_conv = tf.nn.relu(features=bias)


#### 2. 池化样例

In [3]:
# 池化有两种常用的池化
# tf.nn.max_pool最大值池化
# tf.nn.avg_pool均值池化

# ksize为池化层过滤器filter的尺寸，过滤器的头尾两个维度必须等于1
# 实际中使用最多的尺寸是[1,2,2,1]或者[1,3,3,1]
# strides为池化层过滤器步长
pool = tf.nn.max_pool(
    value=actived_conv, 
    ksize=[1,3,3,1], 
    strides=[1,2,2,1], 
    padding='SAME'
)


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(biases))
    print(sess.run(filter_weight))

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1
  0.1]
[[[[ 0.02168818  0.01285435 -0.06766106 ..., -0.07072296  0.06588759
     0.15056656]
   [ 0.05243896 -0.17751265  0.00194047 ..., -0.02968038  0.09518041
     0.02195903]
   [ 0.09532433 -0.08311255 -0.05465999 ...,  0.07416227 -0.06217621
    -0.16228133]]

  [[-0.04648024 -0.16522548 -0.0372405  ...,  0.05240355 -0.00154709
     0.07768789]
   [-0.19078499 -0.05449989  0.07165533 ..., -0.01213859 -0.04813964
    -0.02143265]
   [-0.11788438 -0.06821714 -0.14636274 ..., -0.00467992  0.10337502
     0.02416306]]

  [[ 0.10671563  0.00708553  0.08384535 ...,  0.03588026  0.07476931
     0.05384065]
   [-0.00400845 -0.01078473 -0.12714158 ..., -0.18333089  0.14854972
    -0.03532339]
   [-0.15494664 -0.01021983  0.01156669 ...,  0.041296   -0.02286952
    -0.14389446]]

  [[ 0.09224495  0.01622674 -0.06678429 ...,  0.0276334  -0.07222521
     0.06384867]
   [ 0.00606321  0.05197761  0.03456064 ..., -0.010

#### LENET-5模型

```python
input: [width=32, height=32, color=1]

conv1:
filter: [width=5, height=5, input-dims=1, output-dims=6]
stride: [x-stride=1, y-stride=1]
output: [width=(32-5+1)/1=28, height=(32-5+1)/1=28, output-dims=6]

pool1:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=28/2=14, height=28/2=14, output-dims=6]

conv2:
filter: [width=5, height=5, input-dims=6, output-dims=16]
stride: [x-stride=1, y-stride=1]
output: [width=(14-5+1)/1=10, height=(14-5+1)/1=10, output-dims=16]

pool2:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=10/2=5, height=10/2=5, output-dims=16]

full-connect1:
w: [5*5*16=400, 120]
output: [120]

full-connect2:
w: [120, 84]
output: [84]

full-connect3:
w: [84, 10]
output: [10]
```